# Notebook to update list of indicators in README.md

In [4]:
import re
import shutil

import pandas as pd

from mintalib import core

from config import ROOTDIR


In [5]:

def get_info(func):
    """ information about function """

    info = dict(Name=func.__qualname__)

    doc = func.__doc__ or ""
    description = doc.strip().partition("\n")[0]
    if description is not None:
        info.update(Description=description)

    return info


def list_functions():
    """ list functions """

    result = [v for k, v in vars(core).items()
              if k.isupper() and callable(v)]

    result = [get_info(f) for f in result]

    result = pd.DataFrame(result).set_index('Name')

    result = result.sort_index()

    return result


In [6]:
functions = list_functions()
functions


,Description
Name,
ADX,Average Directional Index
ATR,Average True Range
AVGPRICE,Average Price
BBANDS,Bollinger Bands
BOP,Balance of Power
CCI,Commodity Channel Index
CMF,Chaikin Money Flow
CROSSOVER,Cross Over
CROSSUNDER,Cross Under


In [7]:
help(core.SAR)


Help on cython_function_or_method in module mintalib.core:

SAR(prices, afs: 'float' = 0.02, maxaf: 'float' = 0.2)
    Parabolic Stop and Reverse


In [8]:
def backup_readme(verbose=True):
    readme = ROOTDIR.joinpath("README.md")
    if ROOTDIR.joinpath("archive").exists():
        backup = ROOTDIR.joinpath("archive/README.bak")
    else:
        backup = ROOTDIR.joinpath("README.bak")

    if verbose:
        print(f"Backing up {readme.name} ...")

    shutil.copy(readme, backup)


backup_readme()


Backing up README.md ...


In [9]:

def update_readme(verbose=True):
    title = "## List of Indicators\n"
    table = list_functions().to_markdown()
    repl = title + "\n" + table + "\n\n\n"

    # flags m=multiline, s=dotall
    pattern = r"(?ms)(^[#]+ List of (Functions|Indicators)\n[^#]+)"

    readme = ROOTDIR.joinpath("README.md")
    contents = readme.read_text()

    output, count = re.subn(pattern, repl, contents)

    if count != 1:
        raise RuntimeError("Cound not locate list of funtions")

    if verbose:
        print(f"Updating {readme.name} ...")

    return readme.write_text(output)



update_readme()



Updating README.md ...


8229